In [1]:
import polars as pl
import pandas as pd
import numpy as np
import duckdb

In [2]:
sales = pl.read_csv("data/sales.csv")
vendor_invoice = pl.read_csv("data/vendor_invoice.csv")
purchase = pl.read_csv("data/purchases.csv")
purchase_price = pl.read_csv("data/purchase_prices.csv", infer_schema_length=None)
begin_inventory = pl.read_csv("data/begin_inventory.csv")
end_inventory = pl.read_csv("data/end_inventory.csv")

In [3]:
vendor_invoice.head(3)

VendorNumber,VendorName,InvoiceDate,PONumber,PODate,PayDate,Quantity,Dollars,Freight,Approval
i64,str,str,i64,str,str,i64,f64,f64,str
105,"""ALTAMAR BRANDS LLC ""","""2024-01-04""",8124,"""2023-12-21""","""2024-02-16""",6,214.26,3.47,null
4466,"""AMERICAN VINTAGE BEVERAGE ""","""2024-01-07""",8137,"""2023-12-22""","""2024-02-21""",15,140.55,8.57,null
388,"""ATLANTIC IMPORTING COMPANY ""","""2024-01-09""",8169,"""2023-12-24""","""2024-02-16""",5,106.6,4.61,null


In [4]:
sales.head(3)

InventoryId,Store,Brand,Description,Size,SalesQuantity,SalesDollars,SalesPrice,SalesDate,Volume,Classification,ExciseTax,VendorNo,VendorName
str,i64,i64,str,str,i64,f64,f64,str,f64,i64,f64,i64,str
"""1_HARDERSFIELD_1004""",1,1004,"""Jim Beam w/2 Rocks Glasses""","""750mL""",1,16.49,16.49,"""2024-01-01""",750.0,1,0.79,12546,"""JIM BEAM BRANDS COMPANY """
"""1_HARDERSFIELD_1004""",1,1004,"""Jim Beam w/2 Rocks Glasses""","""750mL""",2,32.98,16.49,"""2024-01-02""",750.0,1,1.57,12546,"""JIM BEAM BRANDS COMPANY """
"""1_HARDERSFIELD_1004""",1,1004,"""Jim Beam w/2 Rocks Glasses""","""750mL""",1,16.49,16.49,"""2024-01-03""",750.0,1,0.79,12546,"""JIM BEAM BRANDS COMPANY """


In [5]:
purchase.head(3)

InventoryId,Store,Brand,Description,Size,VendorNumber,VendorName,PONumber,PODate,ReceivingDate,InvoiceDate,PayDate,PurchasePrice,Quantity,Dollars,Classification
str,i64,i64,str,str,i64,str,i64,str,str,str,str,f64,i64,f64,i64
"""69_MOUNTMEND_8412""",69,8412,"""Tequila Ocho Plata Fresno""","""750mL""",105,"""ALTAMAR BRANDS LLC """,8124,"""2023-12-21""","""2024-01-02""","""2024-01-04""","""2024-02-16""",35.71,6,214.26,1
"""30_CULCHETH_5255""",30,5255,"""TGI Fridays Ultimte Mudslide""","""1.75L""",4466,"""AMERICAN VINTAGE BEVERAGE """,8137,"""2023-12-22""","""2024-01-01""","""2024-01-07""","""2024-02-21""",9.35,4,37.4,1
"""34_PITMERDEN_5215""",34,5215,"""TGI Fridays Long Island Iced""","""1.75L""",4466,"""AMERICAN VINTAGE BEVERAGE """,8137,"""2023-12-22""","""2024-01-02""","""2024-01-07""","""2024-02-21""",9.41,5,47.05,1


In [6]:
purchase_price.head(3)

Brand,Description,Price,Size,Volume,Classification,PurchasePrice,VendorNumber,VendorName
i64,str,f64,str,str,i64,f64,i64,str
58,"""Gekkeikan Black & Gold Sake""",12.99,"""750mL""","""750""",1,9.28,8320,"""SHAW ROSS INT L IMP LTD """
62,"""Herradura Silver Tequila""",36.99,"""750mL""","""750""",1,28.67,1128,"""BROWN-FORMAN CORP """
63,"""Herradura Reposado Tequila""",38.99,"""750mL""","""750""",1,30.46,1128,"""BROWN-FORMAN CORP """


In [7]:
begin_inventory.head(2)

InventoryId,Store,City,Brand,Description,Size,onHand,Price,startDate
str,i64,str,i64,str,str,i64,f64,str
"""1_HARDERSFIELD_58""",1,"""HARDERSFIELD""",58,"""Gekkeikan Black & Gold Sake""","""750mL""",8,12.99,"""2024-01-01"""
"""1_HARDERSFIELD_60""",1,"""HARDERSFIELD""",60,"""Canadian Club 1858 VAP""","""750mL""",7,10.99,"""2024-01-01"""


In [8]:
begin_inventory.tail(2)

InventoryId,Store,City,Brand,Description,Size,onHand,Price,startDate
str,i64,str,i64,str,str,i64,f64,str
"""79_BALLYMENA_90011""",79,"""BALLYMENA""",90011,"""Ch Pichon Longville 12 Pauil""","""750mL""",12,144.99,"""2024-01-01"""
"""79_BALLYMENA_90089""",79,"""BALLYMENA""",90089,"""Ch Lynch Bages 12 Pauillac""","""750mL""",24,119.99,"""2024-01-01"""


In [9]:
end_inventory.head(2)

InventoryId,Store,City,Brand,Description,Size,onHand,Price,endDate
str,i64,str,i64,str,str,i64,f64,str
"""1_HARDERSFIELD_58""",1,"""HARDERSFIELD""",58,"""Gekkeikan Black & Gold Sake""","""750mL""",11,12.99,"""2024-12-31"""
"""1_HARDERSFIELD_62""",1,"""HARDERSFIELD""",62,"""Herradura Silver Tequila""","""750mL""",7,36.99,"""2024-12-31"""


In [10]:
end_inventory.tail(2)

InventoryId,Store,City,Brand,Description,Size,onHand,Price,endDate
str,i64,str,i64,str,str,i64,f64,str
"""81_PEMBROKE_90090""",81,"""PEMBROKE""",90090,"""Ch Lafite Rothschild 12""","""750mL""",3,649.99,"""2024-12-31"""
"""81_PEMBROKE_90604""",81,"""PEMBROKE""",90604,"""Ch Lynch Bages Pauilac""","""750mL""",2,119.99,"""2024-12-31"""


In [11]:
begin_inventory_rows = len(begin_inventory)
end_inventory_rows = len(end_inventory)
print(begin_inventory_rows)
print(end_inventory_rows)


206529
224489


In [12]:
df = pd.DataFrame(begin_inventory)
start_begin_inv_index = (begin_inventory_rows // 2) + 500
print(start_begin_inv_index)
bigin_inventory_mid_5_rows = df.iloc[start_begin_inv_index : start_begin_inv_index + 5]
bigin_inventory_mid_5_rows


103764


,0,1,2,3,4,5,6,7,8
103764,43_WOLFORD_3364,43,WOLFORD,3364,Absolut Ruby Red,1.75L,4,24.99,2024-01-01
103765,43_WOLFORD_3366,43,WOLFORD,3366,Tanqueray Rangpur Gin,1.75L,6,33.99,2024-01-01
103766,43_WOLFORD_3367,43,WOLFORD,3367,Pinnacle Pomegranate Vodka,750mL,10,9.99,2024-01-01
103767,43_WOLFORD_3368,43,WOLFORD,3368,Pinnacle Grape Vodka,1.75L,14,16.99,2024-01-01
103768,43_WOLFORD_3385,43,WOLFORD,3385,Pinnacle Strawbry Kiwi Vodka,1.75L,6,16.99,2024-01-01


In [13]:
dfend = pd.DataFrame(end_inventory)
start_end_inv_index = (end_inventory_rows // 2) + 500
print(start_end_inv_index)
end_inventory_mid_5_rows = dfend.iloc[start_end_inv_index : start_end_inv_index + 5]
end_inventory_mid_5_rows


112744


,0,1,2,3,4,5,6,7,8
112744,45_VERITAS_2586,45,VERITAS,2586,Jameson Irish Whiskey,50mL,72,2.99,2024-12-31
112745,45_VERITAS_2587,45,VERITAS,2587,Jameson Irish Whiskey,375mL,19,18.99,2024-12-31
112746,45_VERITAS_2589,45,VERITAS,2589,Jameson Irish Whiskey,1.75L,34,37.99,2024-12-31
112747,45_VERITAS_2595,45,VERITAS,2595,Clontarf Irish Whiskey,750mL,16,16.99,2024-12-31
112748,45_VERITAS_2606,45,VERITAS,2606,Glenmorangie Original 18Yr,750mL,4,99.99,2024-12-31


In [14]:
unique_bigin_inventory_date = begin_inventory['startDate'].unique()
unique_bigin_inventory_date

startDate
str
"""2024-01-01"""


In [15]:
unique_end_inventory_date = end_inventory['endDate'].unique()
unique_end_inventory_date

endDate
str
"""2024-12-31"""


In [16]:
vendor_list = [105]
brand_list = [60]

# Filter from Purchase_price
filter_purchase_price = purchase_price.filter(
    pl.col('VendorNumber').is_in(vendor_list) | 
    pl.col('Brand').is_in(brand_list)
    )
filter_purchase_price

Brand,Description,Price,Size,Volume,Classification,PurchasePrice,VendorNumber,VendorName
i64,str,f64,str,str,i64,f64,i64,str
8412,"""Tequila Ocho Plata Fresno""",49.99,"""750mL""","""750""",1,35.71,105,"""ALTAMAR BRANDS LLC """
8419,"""Ocho Tequila Plata Corrales""",6.99,"""100mL""","""100""",1,5.3,105,"""ALTAMAR BRANDS LLC """
60,"""Canadian Club 1858 VAP""",10.99,"""750mL""","""750""",1,7.4,12546,"""JIM BEAM BRANDS COMPANY """
2529,"""Right Gin""",29.99,"""750mL""","""750""",1,23.25,105,"""ALTAMAR BRANDS LLC """


In [17]:
filter_purchase = purchase.filter(
    pl.col('VendorNumber').is_in(vendor_list) |
    pl.col('Brand').is_in(brand_list)
    )
filter_purchase.head(5)

InventoryId,Store,Brand,Description,Size,VendorNumber,VendorName,PONumber,PODate,ReceivingDate,InvoiceDate,PayDate,PurchasePrice,Quantity,Dollars,Classification
str,i64,i64,str,str,i64,str,i64,str,str,str,str,f64,i64,f64,i64
"""69_MOUNTMEND_8412""",69,8412,"""Tequila Ocho Plata Fresno""","""750mL""",105,"""ALTAMAR BRANDS LLC """,8124,"""2023-12-21""","""2024-01-02""","""2024-01-04""","""2024-02-16""",35.71,6,214.26,1
"""66_EANVERNESS_8412""",66,8412,"""Tequila Ocho Plata Fresno""","""750mL""",105,"""ALTAMAR BRANDS LLC """,8234,"""2023-12-29""","""2024-01-04""","""2024-01-17""","""2024-02-22""",35.71,6,214.26,1
"""15_WANBORNE_8412""",15,8412,"""Tequila Ocho Plata Fresno""","""750mL""",105,"""ALTAMAR BRANDS LLC """,8234,"""2023-12-29""","""2024-01-07""","""2024-01-17""","""2024-02-22""",35.71,6,214.26,1
"""66_EANVERNESS_8412""",66,8412,"""Tequila Ocho Plata Fresno""","""750mL""",105,"""ALTAMAR BRANDS LLC """,8836,"""2024-02-09""","""2024-02-18""","""2024-02-22""","""2024-03-31""",35.71,6,214.26,1
"""10_HORNSEY_8412""",10,8412,"""Tequila Ocho Plata Fresno""","""750mL""",105,"""ALTAMAR BRANDS LLC """,8836,"""2024-02-09""","""2024-02-17""","""2024-02-22""","""2024-03-31""",35.71,6,214.26,1


In [18]:
filter_invoice = vendor_invoice.filter(pl.col('PONumber') == 13557)
filter_invoice

VendorNumber,VendorName,InvoiceDate,PONumber,PODate,PayDate,Quantity,Dollars,Freight,Approval
i64,str,str,i64,str,str,i64,f64,f64,str
90051,"""FULCHINO VINEYARD INC ""","""2025-01-07""",13557,"""2024-12-18""","""2025-02-10""",59,724.22,3.84,null


In [19]:
filter_sales = sales.filter(pl.col('Brand').is_in(brand_list))
filter_sales

InventoryId,Store,Brand,Description,Size,SalesQuantity,SalesDollars,SalesPrice,SalesDate,Volume,Classification,ExciseTax,VendorNo,VendorName
str,i64,i64,str,str,i64,f64,f64,str,f64,i64,f64,i64,str
"""1_HARDERSFIELD_60""",1,60,"""Canadian Club 1858 VAP""","""750mL""",1,10.99,10.99,"""2024-01-03""",750.0,1,0.79,12546,"""JIM BEAM BRANDS COMPANY """
"""1_HARDERSFIELD_60""",1,60,"""Canadian Club 1858 VAP""","""750mL""",2,21.98,10.99,"""2024-01-12""",750.0,1,1.57,12546,"""JIM BEAM BRANDS COMPANY """
"""1_HARDERSFIELD_60""",1,60,"""Canadian Club 1858 VAP""","""750mL""",1,10.99,10.99,"""2024-01-19""",750.0,1,0.79,12546,"""JIM BEAM BRANDS COMPANY """
"""1_HARDERSFIELD_60""",1,60,"""Canadian Club 1858 VAP""","""750mL""",1,10.99,10.99,"""2024-01-23""",750.0,1,0.79,12546,"""JIM BEAM BRANDS COMPANY """
"""1_HARDERSFIELD_60""",1,60,"""Canadian Club 1858 VAP""","""750mL""",2,21.98,10.99,"""2024-01-29""",750.0,1,1.57,12546,"""JIM BEAM BRANDS COMPANY """
…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""81_PEMBROKE_60""",81,60,"""Canadian Club 1858 VAP""","""750mL""",18,197.82,10.99,"""2024-12-21""",750.0,1,14.17,12546,"""JIM BEAM BRANDS COMPANY """
"""8_ALNERWICK_60""",8,60,"""Canadian Club 1858 VAP""","""750mL""",2,19.98,9.99,"""2024-12-02""",750.0,1,1.57,12546,"""JIM BEAM BRANDS COMPANY """
"""8_ALNERWICK_60""",8,60,"""Canadian Club 1858 VAP""","""750mL""",1,9.99,9.99,"""2024-12-04""",750.0,1,0.79,12546,"""JIM BEAM BRANDS COMPANY """


In [20]:
store = [2]
begin_inventory.filter(
    pl.col('Brand').is_in(brand_list) |
    pl.col('Store').is_in(store)
    )

InventoryId,Store,City,Brand,Description,Size,onHand,Price,startDate
str,i64,str,i64,str,str,i64,f64,str
"""1_HARDERSFIELD_60""",1,"""HARDERSFIELD""",60,"""Canadian Club 1858 VAP""","""750mL""",7,10.99,"""2024-01-01"""
"""2_ASHBORNE_60""",2,"""ASHBORNE""",60,"""Canadian Club 1858 VAP""","""750mL""",5,10.99,"""2024-01-01"""
"""2_ASHBORNE_62""",2,"""ASHBORNE""",62,"""Herradura Silver Tequila""","""750mL""",6,36.99,"""2024-01-01"""
"""2_ASHBORNE_63""",2,"""ASHBORNE""",63,"""Herradura Reposado Tequila""","""750mL""",8,38.99,"""2024-01-01"""
"""2_ASHBORNE_72""",2,"""ASHBORNE""",72,"""No. 3 London Dry Gin""","""750mL""",7,34.99,"""2024-01-01"""
…,…,…,…,…,…,…,…,…
"""74_PAENTMARWY_60""",74,"""PAENTMARWY""",60,"""Canadian Club 1858 VAP""","""750mL""",4,10.99,"""2024-01-01"""
"""75_BARNCOMBE_60""",75,"""BARNCOMBE""",60,"""Canadian Club 1858 VAP""","""750mL""",12,10.99,"""2024-01-01"""
"""76_DONCASTER_60""",76,"""DONCASTER""",60,"""Canadian Club 1858 VAP""","""750mL""",65,10.99,"""2024-01-01"""


In [21]:
end_inventory.filter(
    pl.col('Brand').is_in(brand_list) |
    pl.col('Store').is_in(store)
    )

InventoryId,Store,City,Brand,Description,Size,onHand,Price,endDate
str,i64,str,i64,str,str,i64,f64,str
"""2_ASHBORNE_60""",2,"""ASHBORNE""",60,"""Canadian Club 1858 VAP""","""750mL""",1,10.99,"""2024-12-31"""
"""2_ASHBORNE_62""",2,"""ASHBORNE""",62,"""Herradura Silver Tequila""","""750mL""",3,36.99,"""2024-12-31"""
"""2_ASHBORNE_63""",2,"""ASHBORNE""",63,"""Herradura Reposado Tequila""","""750mL""",6,38.99,"""2024-12-31"""
"""2_ASHBORNE_77""",2,"""ASHBORNE""",77,"""Three Olives Espresso Vodka""","""750mL""",46,14.99,"""2024-12-31"""
"""2_ASHBORNE_79""",2,"""ASHBORNE""",79,"""Three Olives Loopy Vodka""","""750mL""",13,14.99,"""2024-12-31"""
…,…,…,…,…,…,…,…,…
"""64_CESTERFIELD_60""",64,"""CESTERFIELD""",60,"""Canadian Club 1858 VAP""","""750mL""",2,10.99,"""2024-12-31"""
"""67_EANVERNESS_60""",67,"""EANVERNESS""",60,"""Canadian Club 1858 VAP""","""750mL""",16,10.99,"""2024-12-31"""
"""70_KELD_60""",70,"""KELD""",60,"""Canadian Club 1858 VAP""","""750mL""",1,10.99,"""2024-12-31"""


In [22]:
# Dimension - Brand - Purchase and Sales
dim_brand_purchase_sales = (
    pl.concat([
        sales.select(pl.col('Brand')),
        purchase.select(pl.col('Brand')),
    ]).unique().with_row_index('Brand_ID', offset=1)
)

In [23]:
# Dimension - Vendor
dim_vendor = (
    pl.concat([
        sales.select(pl.col('VendorNo').alias('VendorNumber')),
        vendor_invoice.select(pl.col('VendorNumber')),
        purchase.select(pl.col('VendorNumber')),
        purchase_price.select(pl.col('VendorNumber'))
    ]).unique().with_row_index('vendor_id', offset=1)
)

In [24]:
# Dimension - Store
dim_store = (
    pl.concat([
        sales.select(pl.col('Store')),
        purchase.select(pl.col('Store')),
        begin_inventory.select(pl.col('Store')),
        end_inventory.select(pl.col('Store'))
    ]).unique().with_row_index('store_id', offset=1)
)

In [25]:
# Dimension - Inventory ID
dim_inventory = (
    pl.concat([
        sales.select(pl.col('InventoryId')),
        purchase.select(pl.col('InventoryId')),
        begin_inventory.select(pl.col('InventoryId')),
        end_inventory.select(pl.col('InventoryId'))
    ]).unique().with_row_index('id', offset=1)
)

In [26]:
# Dimension - Brand Inventory
dim_brand_inventory = (
    pl.concat([
        begin_inventory.select(pl.col('Brand')),
        end_inventory.select(pl.col('Brand'))
    ]).unique().with_row_index('id', offset=1)
)

In [27]:
unique_value_count = dim_brand_inventory.select(pl.col("Brand").n_unique().sum()).item()
print('Unique count: ', unique_value_count)
print("*******")
duplicate_count = dim_brand_inventory.select(pl.col("Brand").is_duplicated().sum()).item()
print('Duplicate count: ' ,duplicate_count)


Unique count:  10759
*******
Duplicate count:  0
